## Abusing multiple dispatch

So now we can sacrifice speed for magic! Note that this code is slower than the previous examples:

In [1]:
# Terminate recursion
fib_3(::Type{Val{1}}) = 1
fib_3(::Type{Val{2}}) = 1

# Translate integer input into its own unique type
fib_3(n::Integer) = fib_3(Val{n})

# Recursive definitionin the dispatcher
fib_3(n::Type{Val{N}}) where N = fib_3(N-1) + fib_3(N-2)

fib_3 (generic function with 4 methods)

In [2]:
fib_3(32)

2178309

In [3]:
methods(fib_3)

# 4 methods for generic function "fib_3":
[1] fib_3(::Type{Val{1}}) in Main at In[1]:5
[2] fib_3(::Type{Val{2}}) in Main at In[1]:6
[3] fib_3(n::Integer) in Main at In[1]:9
[4] fib_3(n::Type{Val{N}}) where N in Main at In[1]:12

In [4]:
@code_llvm fib_3(Val{32})

;  @ In[1]:12 within `fib_3'
define nonnull {}* @japi1_fib_3_1448({}* %0, {}** %1, i32 %2) #0 {
top:
  %3 = alloca [2 x {}*], align 8
  %gcframe2 = alloca [4 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %3, i64 0, i64 0
  %4 = bitcast [4 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i32(i8* nonnull align 16 dereferenceable(32) %4, i8 0, i32 32, i1 false)
  %5 = alloca {}**, align 8
  store volatile {}** %1, {}*** %5, align 8
  %6 = call {}*** inttoptr (i64 4478096240 to {}*** ()*)() #4
;  @ In[1]:12 within `fib_3' @ In[1]:9
  %7 = bitcast [4 x {}*]* %gcframe2 to i64*
  store i64 8, i64* %7, align 16
  %8 = bitcast {}*** %6 to i64*
  %9 = load i64, i64* %8, align 8
  %10 = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 1
  %11 = bitcast {}** %10 to i64*
  store i64 %9, i64* %11, align 8
  %12 = bitcast {}*** %6 to {}***
  store {}** %gcframe2.sub

In [5]:
@code_llvm fib_3(Val{1})

;  @ In[1]:5 within `fib_3'
define i64 @julia_fib_3_1482() {
top:
  ret i64 1
}


In [8]:
@code_llvm fib_3(Val{2})

;  @ In[1]:6 within `fib_3'
define i64 @julia_fib_3_1626() {
top:
  ret i64 1
}


In [9]:
@code_llvm fib_3(Val{3})

;  @ In[1]:12 within `fib_3'
define i64 @julia_fib_3_1628() {
top:
  ret i64 2
}


In [10]:
@code_llvm fib_3(Val{4})

;  @ In[1]:12 within `fib_3'
define nonnull {}* @japi1_fib_3_1630({}* %0, {}** %1, i32 %2) #0 {
top:
  %3 = alloca [2 x {}*], align 8
  %gcframe2 = alloca [3 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %3, i64 0, i64 0
  %4 = bitcast [3 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i32(i8* nonnull align 16 dereferenceable(24) %4, i8 0, i32 24, i1 false)
  %5 = alloca {}**, align 8
  store volatile {}** %1, {}*** %5, align 8
  %6 = call {}*** inttoptr (i64 4478096240 to {}*** ()*)() #4
;  @ In[1]:12 within `fib_3' @ In[1]:9
  %7 = bitcast [3 x {}*]* %gcframe2 to i64*
  store i64 4, i64* %7, align 16
  %8 = bitcast {}*** %6 to i64*
  %9 = load i64, i64* %8, align 8
  %10 = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe2, i64 0, i64 1
  %11 = bitcast {}** %10 to i64*
  store i64 %9, i64* %11, align 8
  %12 = bitcast {}*** %6 to {}***
  store {}** %gcframe2.sub

In [7]:
using BenchmarkTools

In [6]:
@benchmark fib_3(10)

LoadError: LoadError: UndefVarError: @benchmark not defined
in expression starting at In[6]:1